In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 7b686a7d-cb21-46f2-8961-e624eee5cf4f
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session 7b686a7d-cb21-46f2-8961-e624eee5cf4f to get into ready status...
Session 7b686a7d-cb21-46f2-8961-e624eee5cf4f ha

In [24]:
YEAR = "2021"
MONTH = "12"

In [25]:
df = spark.read.parquet(f"s3://finalproject-9458/data/specific/{YEAR}-{MONTH}")

In [26]:
df.show(5)

+-----+---------+---------+--------------------+--------+---------+----------+--------------+------------+----------+---+
|index|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|invoice_date|      date|day|
+-----+---------+---------+--------------------+--------+---------+----------+--------------+------------+----------+---+
|10144|   537226|    22811|SET OF 6 T-LIGHTS...|       6|        3|      null|United Kingdom|  2021-12-06|2021-12-06|  6|
|10145|   537226|    21713|CITRONELLA CANDLE...|       8|        2|      null|United Kingdom|  2021-12-06|2021-12-06|  6|
|10146|   537226|    22927|GREEN GIANT GARDE...|       2|        6|      null|United Kingdom|  2021-12-06|2021-12-06|  6|
|10147|   537226|    20802|SMALL GLASS SUNDA...|       6|        2|      null|United Kingdom|  2021-12-06|2021-12-06|  6|
|10148|   537226|    22052|VINTAGE CARAVAN G...|      25|        0|      null|United Kingdom|  2021-12-06|2021-12-06|  6|
+-----+---------+-------

In [27]:
df.groupby("day").count().show()

+---+-----+
|day|count|
+---+-----+
|  1| 3108|
|  6| 3878|
| 17| 3115|
| 16| 1790|
| 20| 1763|
| 21| 1586|
| 10| 2758|
|  9| 2891|
|  7| 2963|
|  3| 2202|
|  2| 2109|
| 14| 2087|
|  5| 2725|
| 13| 2283|
|  8| 2647|
| 22|  291|
| 19|  522|
| 12| 1451|
| 23|  963|
| 15| 1349|
+---+-----+


In [28]:
df = df.filter((df["Quantity"] > 0) & (df["UnitPrice"] > 0))

In [29]:
total_product_rdd = df.rdd.map(lambda row: (row.StockCode, (row.Quantity, row.Quantity * row.UnitPrice)))

In [30]:
total_product_rdd = total_product_rdd.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
df_total_product = spark.createDataFrame(total_product_rdd.map(lambda x: (x[0], x[1][0], x[1][1])), schema=["ProductStockCode", "TotalQuantity", "TotalAmount"])

In [31]:
df_total_product.show()

+----------------+-------------+--------------------+
|ProductStockCode|TotalQuantity|         TotalAmount|
+----------------+-------------+--------------------+
|           22111|          958|5234.000000000000...|
|           22445|          119|369.0000000000000...|
|           82486|          230|1758.000000000000...|
|           22569|          286|1168.000000000000...|
|           21080|          856|980.0000000000000...|
|           22837|         1085|5286.000000000000...|
|           21479|         1407|7934.000000000000...|
|           22283|           30|344.0000000000000...|
|          84997C|          183|676.0000000000000...|
|           20681|           45|171.0000000000000...|
|           22149|          337|909.0000000000000...|
|           22751|          127|480.0000000000000...|
|           84992|          749|749.0000000000000...|
|           22834|         3502|4955.000000000000...|
|           22654|          324|2904.000000000000...|
|           22594|          

In [35]:
df_total_product.coalesce(1).write.mode("overwrite").csv(f"s3://finalproject-9458/data/reporting/{YEAR}-{MONTH}/total_per_product/")

In [ ]:
df.coalesce(1).write.mode("overwrite").csv(f"s3://finalproject-9458/data/reporting/{YEAR}-{MONTH}/general/")